In [12]:
# %pip install cowdao-cowpy==1.0.0 seaborn "web3>=6.0,<7.0" "safe-eth-py"
# # Uninstall any conflicting versions
# %pip uninstall web3 safe-eth-py prediction-market-agent-tooling -y

# # Install the compatible versions from prediction-market-agent's lock file
# %pip install "web3>=6.15.1,<7.0.0"
# %pip install "safe-eth-py>=6.0.0b41,<7.0.0"

# # Reinstall prediction-market-agent-tooling to pick up the right web3 version
# %pip install "prediction-market-agent-tooling>=0.65.10"
%pip list 

Package                                  Version            Editable project location
---------------------------------------- ------------------ ------------------------------------------------
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.10.11
aiolimiter                               1.2.1
aiosignal                                1.4.0
altair                                   5.5.0
altex                                    0.2.0
annotated-types                          0.7.0
anthropic                                0.64.0
anyio                                    4.10.0
ape-foundry                              0.8.7
appdirs                                  1.4.4
argcomplete                              3.6.2
argon2-cffi                              25.1.0
argon2-cffi-bindings                     25.1.0
arrow                                    1.3.0
art                                      6.5
a

In [13]:
import sys
import os

# Add the prediction-market-agent directory to Python path
sys.path.insert(0, '/home/valden/Work/gnosis/prediction-market-agent')


import nest_asyncio
nest_asyncio.apply()

from IPython.display import display, HTML
import pandas as pd
import json

from datetime import datetime, timedelta, timezone
from functools import partial

import pandas as pd
from prediction_market_agent.development_tools.prophet_agent_tester import (
    ProphetAgentTester,
)

from prediction_market_agent.utils import APIKeys
from prediction_market_agent_tooling.deploy.betting_strategy import BinaryKellyBettingStrategy
from prediction_market_agent_tooling.gtypes import USD
from prediction_market_agent_tooling.loggers import logger
from prediction_market_agent_tooling.markets.agent_market import AgentMarket
from prediction_prophet.autonolas.research import (
    FIELDS_DESCRIPTIONS,
    PREDICTION_PROMPT,
    LogprobsParser,
)
from prediction_prophet.functions.research import Research, research

from prediction_prophet.autonolas.research import Prediction
from prediction_prophet.autonolas.research import Prediction as PredictionProphet
from prediction_prophet.autonolas.research import (
    clean_completion_json,
    fields_dict_to_bullet_list,
    list_to_list_str,
)

import json
import ast
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.settings import ModelSettings

from prediction_market_agent.agents.utils import get_maximum_possible_bet_amount
from prediction_market_agent_tooling.tools.openai_utils import get_openai_provider

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

In [14]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
GPT_4O_MODEL = "DeployablePredictionProphetGPT4oAgent"
DATASET_PATH_07 = "/home/valden/Documents/DeployablePredictionProphetGPT4oAgent_2025-07-01_2025-07-30.csv"
df_07 = pd.read_csv(DATASET_PATH_07)
df_07 = df_07[df_07["agent_name"] == "DeployablePredictionProphetGPT4oAgent"]
df_07["market_close_time"] = pd.to_datetime(df_07["market_close_time"])

# Take trades only from June and July
df_07 = df_07[df_07["market_close_time"].dt.month.isin([6,7,8])]
# filter only witn bet_json
df_07 = df_07[df_07["bet_json"].notna()]

In [16]:
DATASET_PATH_05 = "/home/valden/Documents/DeployablePredictionProphetGPT4oAgent_2025-05-01_2025-05-30.csv"
df_05 = pd.read_csv(DATASET_PATH_05)
df_05 = df_05[df_05["agent_name"] == "DeployablePredictionProphetGPT4oAgent"]
df_05["market_close_time"] = pd.to_datetime(df_05["market_close_time"])

# Take trades only from June and July
df_05 = df_05[df_05["market_close_time"].dt.month.isin([4,5])]
# filter only witn bet_json
df_05 = df_05[df_05["bet_json"].notna()]

In [17]:


def evaluate_results_dataframe_accurate_fixed(
    df: pd.DataFrame,
    print_individual_metrics: bool = False,
    run_name: str = "DataFrame Evaluation"
) -> dict:
    if df.empty:
        print("Warning: No data to evaluate")
        return None

    total_trades = len(df)
    print(f"Evaluating {total_trades} test results")

    valid_results = df.dropna(subset=['prediction_p_yes', 'market_resolution'])
    if valid_results.empty:
        print("Warning: No valid predictions found")
        return None

    print(f"Valid results: {len(valid_results)}")

    y_true = valid_results['market_resolution'].str.lower()
    valid_results = valid_results.copy()
    valid_results['prediction_p_no'] = 1 - valid_results['prediction_p_yes']

    y_pred_label, y_pred_weight = [], []
    for _, row in valid_results.iterrows():
        p_yes = row['prediction_p_yes']
        p_no = row['prediction_p_no']
        if p_yes > p_no:
            pred_label, pred_weight = 'yes', p_yes
        else:
            pred_label, pred_weight = 'no', p_no
        y_pred_label.append(pred_label)
        y_pred_weight.append(pred_weight)

    binary_prediction_accuracy = [
        1 if true_val == pred_val else 0
        for true_val, pred_val in zip(y_true, y_pred_label)
    ]
    weighted_prediction_accuracy = [
        weight if true_val == pred_val else (1 - weight)
        for true_val, pred_val, weight in zip(y_true, y_pred_label, y_pred_weight)
    ]

    brier_scores = []
    for _, row in valid_results.iterrows():
        actual_outcome = row['market_resolution'].lower()
        p_yes = row['prediction_p_yes']
        p_no = row['prediction_p_no']
        probs = {'yes': p_yes, 'no': p_no}
        brier_score = 0.0
        for outcome, predicted_prob in probs.items():
            true_value = 1.0 if outcome.lower() == actual_outcome else 0.0
            brier_score += (predicted_prob - true_value) ** 2
        brier_scores.append(brier_score)

    avg_brier_score = sum(brier_scores) / len(brier_scores) if brier_scores else 0
    avg_binary_prediction_accuracy = sum(binary_prediction_accuracy) / len(binary_prediction_accuracy)
    avg_weighted_prediction_accuracy = sum(weighted_prediction_accuracy) / len(weighted_prediction_accuracy)

    # Trade metrics
    avg_binary_trade_accuracy = None
    sum_profit_usd = None
    sum_investment_usd = None
    roi = None
    total_buy_trades = 0
    profitable_trades = 0

    def _norm_outcome(x):
        if isinstance(x, bool):
            return 'yes' if x else 'no'
        return str(x).lower() if x is not None else None

    if 'bet_json' in valid_results.columns:
        profits, investments = [], []
        correct, total = 0, 0

        for _, row in valid_results.iterrows():
            bj = row.get('bet_json')
            if pd.isna(bj) or bj in (None, '', 'null'):
                continue
            try:
                bet = json.loads(bj) if isinstance(bj, str) else bj
            except Exception:
                continue

            amt = bet.get('amount', {}).get('value')
            prof = bet.get('profit', {}).get('value')
            b_out = _norm_outcome(bet.get('outcome'))
            m_out = _norm_outcome(bet.get('market_outcome')) or _norm_outcome(row.get('market_resolution'))

            if amt is not None:
                investments.append(float(amt))
                total_buy_trades += 1
            if prof is not None:
                profits.append(float(prof))
                if float(prof) > 0:
                    profitable_trades += 1

            if b_out and m_out:
                total += 1
                if b_out == m_out:
                    correct += 1

        if total > 0:
            avg_binary_trade_accuracy = correct / total
        if investments:
            sum_investment_usd = sum(investments)
        if profits:
            sum_profit_usd = sum(profits)
        if sum_investment_usd and sum_investment_usd > 0 and sum_profit_usd is not None:
            roi = (sum_profit_usd / sum_investment_usd) * 100

    elif 'trades' in valid_results.columns:
        # Fallback: accuracy from first buy trade per row; ROI not computed
        correct = 0
        for _, row in valid_results.iterrows():
            t = row.get('trades')
            if pd.isna(t) or t in (Nprediction_market_agent_market_toolingone, '', '[]'):
                continue
            try:
                trades = ast.literal_eval(t) if isinstance(t, str) else t
            except Exception:
                continue
            buy_trades = [tr for tr in trades if tr.get('trade_type') == 'buy']
            if not buy_trades:
                continue
            first = buy_trades[0]
            total_buy_trades += 1
            trade_outcome = _norm_outcome(first.get('outcome'))
            actual = _norm_outcome(row.get('market_resolution'))
            if trade_outcome and actual and trade_outcome == actual:
                correct += 1
        if total_buy_trades > 0:
            avg_binary_trade_accuracy = correct / total_buy_trades

    metrics = {
        'total_trades': total_trades,
        'valid_trades': len(valid_results),
        'total_buy_trades': total_buy_trades,
        'profitable_trades': profitable_trades,
        'binary_prediction_accuracy': avg_binary_prediction_accuracy,
        'weighted_prediction_accuracy': avg_weighted_prediction_accuracy,
        'binary_trade_accuracy': avg_binary_trade_accuracy,
        'prediction_brier_score': avg_brier_score,
        'profit_usd': sum_profit_usd,
        'investment_usd': sum_investment_usd,
        'roi': roi
    }

    if print_individual_metrics:
        print("\n" + "=" * 50)
        print(f"EVALUATION METRICS {run_name}")
        print("=" * 50)
        print(f"Total Trades: {total_trades}")
        print(f"Valid Trades: {len(valid_results)}")
        print(f"Total Buy Trades: {total_buy_trades}")
        print(f"Profitable Trades: {profitable_trades}")
        print(f"Binary Prediction Accuracy: {avg_binary_prediction_accuracy:.4f}")
        print(f"Weighted Prediction Accuracy: {avg_weighted_prediction_accuracy:.4f}")
        print(f"Brier Score: {avg_brier_score:.4f}")
        if avg_binary_trade_accuracy is not None:
            print(f"Binary Trade Accuracy: {avg_binary_trade_accuracy:.4f}")
            print(f"Total Profit (collateral): {sum_profit_usd:.2f}" if sum_profit_usd is not None else "Total Profit: N/A")
            print(f"Total Investment (collateral): {sum_investment_usd:.2f}" if sum_investment_usd is not None else "Total Investment: N/A")
            print(f"ROI: {roi:.2f}%" if roi is not None else "ROI: N/A")

    return metrics

def calculate_average_confidence(df: pd.DataFrame) -> float:
    return df["prediction_confidence"].mean()

def calculate_p_yes_percentage(df: pd.DataFrame) -> float:
    return df["prediction_p_yes"].mean()

def calculate_average_prediction_decision(df: pd.DataFrame) -> float:
    # Count y and n return percentage y and percentange n
    y_count = df[df["prediction_decision"] == "y"].shape[0] 
    n_count = df[df["prediction_decision"] == "n"].shape[0]
    return y_count / (y_count + n_count), n_count / (y_count + n_count)

def calculate_average_market_resolution(df: pd.DataFrame) -> float:
    # Count y and n return percentage y and percentange n
    y_count = df[df["market_resolution"] == "Yes"].shape[0]
    n_count = df[df["market_resolution"] == "No"].shape[0]
    return y_count / (y_count + n_count), n_count / (y_count + n_count)

In [18]:
print("\n=== EVALUATION WITH FIXED LOGIC ===")
metrics_fixed = evaluate_results_dataframe_accurate_fixed(df_05, print_individual_metrics=True)
metrics_fixed, "average_confidence", calculate_average_confidence(df_05), "average_prediction_decision", calculate_average_prediction_decision(df_05), "average_market_resolution", calculate_average_market_resolution(df_05)


=== EVALUATION WITH FIXED LOGIC ===
Evaluating 1257 test results
Valid results: 1257

EVALUATION METRICS DataFrame Evaluation
Total Trades: 1257
Valid Trades: 1257
Total Buy Trades: 1257
Profitable Trades: 685
Binary Prediction Accuracy: 0.5839
Weighted Prediction Accuracy: 0.5626
Brier Score: 0.5807
Binary Trade Accuracy: 1.0000
Total Profit (collateral): 139.07
Total Investment (collateral): 1677.04
ROI: 8.29%


({'total_trades': 1257,
  'valid_trades': 1257,
  'total_buy_trades': 1257,
  'profitable_trades': 685,
  'binary_prediction_accuracy': 0.5839299920445505,
  'weighted_prediction_accuracy': 0.5625775656324558,
  'binary_trade_accuracy': 1.0,
  'prediction_brier_score': 0.5807373110580759,
  'profit_usd': 139.07129709742554,
  'investment_usd': 1677.0436342170785,
  'roi': 8.292646312828374},
 'average_confidence',
 0.8081543357199681,
 'average_prediction_decision',
 (0.3341288782816229, 0.665871121718377),
 'average_market_resolution',
 (0.34765314240254575, 0.6523468575974543))

In [19]:
print("\n==head(100)= EVALUATION WITH FIXED LOGIC ===")
metrics_fixed = evaluate_results_dataframe_accurate_fixed(df_07, print_individual_metrics=True)
metrics_fixed, calculate_average_confidence(df_07), "Average prediction decision", calculate_average_prediction_decision(df_07), "Average market resolution", calculate_average_market_resolution(df_07)


==head(100)= EVALUATION WITH FIXED LOGIC ===
Evaluating 1971 test results
Valid results: 1971

EVALUATION METRICS DataFrame Evaluation
Total Trades: 1971
Valid Trades: 1971
Total Buy Trades: 1971
Profitable Trades: 701
Binary Prediction Accuracy: 0.6408
Weighted Prediction Accuracy: 0.5847
Brier Score: 0.4983
Binary Trade Accuracy: 1.0000
Total Profit (collateral): -387.75
Total Investment (collateral): 2251.73
ROI: -17.22%


({'total_trades': 1971,
  'valid_trades': 1971,
  'total_buy_trades': 1971,
  'profitable_trades': 701,
  'binary_prediction_accuracy': 0.6407914764079148,
  'weighted_prediction_accuracy': 0.5846626078132907,
  'binary_trade_accuracy': 1.0,
  'prediction_brier_score': 0.49829863013698755,
  'profit_usd': -387.75202042011017,
  'investment_usd': 2251.727907974345,
  'roi': -17.220198721475725},
 0.7990360223236936,
 'Average prediction decision',
 (0.49365804160324706, 0.5063419583967529),
 'Average market resolution',
 (0.3556570268899036, 0.6443429731100964))

In [20]:
def execute_prophet_research(
    use_summaries: bool = False,
    use_tavily_raw_content: bool = False,
    initial_subqueries_limit: int = 20,
    subqueries_limit: int = 5,
    max_results_per_search: int = 5,
    min_scraped_sites: int = 5,
) -> partial[Research]:
    return lambda *args, **kwargs: Research(report="dummy", all_queries=[], reranked_queries=[], websites_to_scrape=[], websites_scraped=[])

def execute_prophet_predict_logprobs(
    include_reasoning: bool = True,
) -> partial[PredictionProphet]:
    return lambda *args, **kwargs: PredictionProphet(decision="y", p_yes=0.5, p_no=0.5, confidence=0.5, info_utility=0.5)

def execute_prophet_predict(
    include_reasoning: bool = True,
) -> partial[PredictionProphet]:
    return lambda *args, **kwargs: PredictionProphet(decision="y", p_yes=0.5, p_no=0.5, confidence=0.5, info_utility=0.5)

def test_kelly_bet_difference(
    dataset_05: pd.DataFrame,
    dataset_07: pd.DataFrame,
    max_trades_to_test_on: int = 3000,
    delay_between_trades: float = 2.0,
    use_old_research: bool = True,
    use_old_prediction: bool = True,
) -> tuple[list, list]:
    dataset_05 = pd.read_csv(dataset_05)
    dataset_07 = pd.read_csv(dataset_07)

    api_keys = APIKeys()
    research_agent = Agent(
        OpenAIModel(
            GPT_4O_MODEL,
            provider=get_openai_provider(api_key=api_keys.openai_api_key),
        ),
        model_settings=ModelSettings(temperature=0.7),
    )
    prediction_agent = Agent(
        OpenAIModel(
            GPT_4O_MODEL,
            provider=get_openai_provider(api_key=api_keys.openai_api_key),
        ),
        model_settings=ModelSettings(
            temperature=0.0, extra_body={"logprobs": True, "top_logprobs": 3}
        ),
    )

    simple_strategy = BinaryKellyBettingStrategy(
        max_position_amount=get_maximum_possible_bet_amount(
            min_=USD(1),
            max_=USD(5),
            trading_balance=USD(10),
        ),
        max_price_impact=0.7,
        force_simplified_calculation=True,
    )

    full_strategy = BinaryKellyBettingStrategy(
        max_position_amount=get_maximum_possible_bet_amount(
            min_=USD(1),
            max_=USD(5),
            trading_balance=USD(10),
        ),
        max_price_impact=0.7,
        force_simplified_calculation=False,
    )

    # Test with simplified Kelly betting
    simple_kelly_bet = ProphetAgentTester(
        prophet_research=execute_prophet_research(),
        prophet_predict=execute_prophet_predict_logprobs(),
        betting_strategy=simple_strategy,
        use_old_research=True,
        use_old_prediction=True,
        max_trades_to_test_on=max_trades_to_test_on,
        run_name="test_simple_kelly_bet",
        mocked_agent_name="DeployablePredictionProphetGPT4oAgent",
        delay_between_trades=delay_between_trades,
        simulate_trades=True,
        bet_only=True
    )

    dataset_05_test_results, dataset_05_trade_logs = simple_kelly_bet.test_prophet_agent(
        dataset_05, research_agent, prediction_agent
    )
    dataset_07_test_results, dataset_07_trade_logs = simple_kelly_bet.test_prophet_agent(
        dataset_07, research_agent, prediction_agent
    )

    return (dataset_05_test_results, dataset_05_trade_logs), (
        dataset_07_test_results,
        dataset_07_trade_logs,
    )

In [21]:
def compare_methods(dataset_05_results, dataset_07_results):
    """Compare logprobs vs standard method performance."""

    dataset_tester = ProphetAgentTester(
        prophet_research=execute_prophet_research(),
        prophet_predict=execute_prophet_predict_logprobs(),
        use_old_research=True,
        use_old_prediction=True,
        run_name="test_simple_kelly_bet",
        mocked_agent_name="DeployablePredictionProphetGPT4oAgent",
        simulate_trades=True,
        bet_only=True
    )

    dataset_05_test_results, dataset_05_trade_logs = dataset_05_results
    dataset_07_test_results, dataset_07_trade_logs = dataset_07_results


    dataset_05_metrics = dataset_tester.evaluate_results(
        dataset_05_test_results, print_individual_metrics=True
    )
    dataset_07_metrics = dataset_tester.evaluate_results(
        dataset_07_test_results, print_individual_metrics=True
    )

    comparison_data = {
        "Method": ["Logprobs", "Standard"],
        "Binary Accuracy": [
            dataset_05_metrics.binary_prediction_accuracy if dataset_05_metrics else 0,
            dataset_07_metrics.binary_prediction_accuracy if dataset_07_metrics else 0,
        ],
        "Weighted Accuracy": [
            dataset_05_metrics.weighted_prediction_accuracy if dataset_05_metrics else 0,
            dataset_07_metrics.weighted_prediction_accuracy if dataset_07_metrics else 0,
        ],
        "Brier Score": [
            dataset_05_metrics.prediction_brier_score if dataset_05_metrics else 0,
            dataset_07_metrics.prediction_brier_score if dataset_07_metrics else 0,
        ],
        "Total Trades": [
            dataset_05_metrics.total_trades if dataset_05_metrics else 0,
            dataset_07_metrics.total_trades if dataset_07_metrics else 0,
        ],
    }

    comparison_df = pd.DataFrame(comparison_data)
    return comparison_df, dataset_05_metrics, dataset_07_metrics

In [22]:
dataset_05_results, dataset_07_results = test_kelly_bet_difference(
    dataset_05=DATASET_PATH_05,
    dataset_07=DATASET_PATH_07,
    max_trades_to_test_on=300,
    delay_between_trades=0,
)
print("Testing completed successfully!")

comparison_df, dataset_05_metrics, dataset_07_metrics = compare_methods(
    dataset_05_results, dataset_07_results
)

2025-08-20 12:19:53.869 | INFO     | prediction_market_agent.development_tools.prophet_agent_tester:test_prophet_agent:116 - Found 545 trades for DeployablePredictionProphetGPT4oAgent, processing 300
2025-08-20 12:19:53.871 | INFO     | prediction_market_agent.development_tools.prophet_agent_tester:test_prophet_agent:128 - Processing trade 1/300: Will the England and Wales Cricket Board officially announce a ban on transgender women from women's cricket by May 7, 2025?
2025-08-20 12:19:54.484 | INFO     | prediction_market_agent.development_tools.prophet_agent_tester:test_prophet_agent:128 - Processing trade 2/300: Will the US Department of Health and Human Services announce new placebo-controlled trial requirements for updated Covid-19 vaccines by May 7, 2025?
2025-08-20 12:19:54.729 | INFO     | prediction_market_agent.development_tools.prophet_agent_tester:test_prophet_agent:128 - Processing trade 3/300: Will the Newquay Zoo-born lynx be officially released into the wild from its en

Testing completed successfully!
